## Facebook Data Collecting (Thu thập dữ liệu Facebook)

### Phương pháp sử dụng
Thu thập dữ liệu sử dụng <a href="https://developers.facebook.com/docs/graph-api/">Facebook Graph API</a> và <a href="https://id.traodoisub.com">API lấy UID Facebook</a> thông qua thư viện 
<a href="https://pypi.org/project/requests/">`requests`</a>

### Cài đặt các thư viện cần thiết

In [ ]:
%pip install facebook_scraper matplotlib requests selenium pandas numpy

In [1]:
import json

import pandas as pd
import requests as rq

##### Cài đặt class FacebookCrawler để có thể dễ dàng trong việc cào dữ liệu

In [9]:
class FacebookCrawler:
    def __init__(self, cookie_path, access_tokens, page_url) -> None:
        self.cookie_path = cookie_path
        self.page_url = "https://facebook.com/{0}".format(
            page_url.replace("/", " ").strip().split()[-1]
        )
        self.id_page = rq.post(
            "https://id.traodoisub.com/api.php", data={"link": self.page_url}
        ).json()["id"]
        self.access_tokens = access_tokens

        self.API_PAGE_POSTS = "https://graph.facebook.com/v18.0/{0}/feed".format(
            self.id_page
        )
        self.API_POST_COMMENTS = "https://graph.facebook.com/v18.0/{0}/comments"

    def export_to_csv(self, data, file_name):
        if ".csv" not in file_name:
            file_name = file_name + ".csv"
        try:
            df = pd.DataFrame(columns=data[0].keys(), index=range(len(data)), data=data)
            df.to_csv("Data/{0}".format(file_name), index=False)
            return True
        except Exception:
            return False

    def export_to_json(self, data, file_name):
        if ".json" not in file_name:
            file_name = file_name + ".json"
        try:
            df = pd.DataFrame(columns=data[0].keys(), index=range(len(data)), data=data)
            with open("Data/{0}".format(file_name), "w") as f:
                json.dump(df.to_json(orient="records", force_ascii=False), f, indent=4)
            return True
        except Exception:
            return False

    def get_page_posts(self, limit_post=100, limit_comment=25):
        idx_token = 0
        self.access_token = self.access_tokens[idx_token]
        _limit_post = 1
        res = rq.get(
            self.API_PAGE_POSTS,
            params={
                "access_token": self.access_token,
                "limit": _limit_post,
                "fields": """
                    created_time.summary(1).as(created_time),
                from.summary(1).as(from),
                updated_time.summary(1).as(updated_time),
                message.summary(1).as(message),
                message_tags.summary(1).as(message_tags),
                type.summary(1).as(type),
                picture.summary(1).as(picture),
                source.summary(1).as(source),
                properties.summary(1).as(properties),
                reactions.type(LIKE).limit(0).summary(1).as(like),
                reactions.type(LOVE).limit(0).summary(1).as(love),
                reactions.type(HAHA).limit(0).summary(1).as(haha),
                reactions.type(WOW).limit(0).summary(1).as(wow),
                reactions.type(ANGRY).limit(0).summary(1).as(angry),
                reactions.type(SAD).limit(0).summary(1).as(sad),
                comments.summary(1).limit("""
                + str(limit_comment)
                + """){message,
                    created_time, 
                    reactions.summary(true).limit(0).as(reactions),
                    from{id,name,first_name,last_name,gender,birthday,location,work}},
                shares.summary(1).as(shares)
            """,
            },
        )
        try:
            _next = res.json()["paging"]["next"]
            yield {"status": True, "data": res.json()["data"]}
        except Exception:
            yield {"status": False, "data": [], "message": res.json()["error"]}
        params = {"limit": _limit_post}
        for i in range(_limit_post, limit_post, _limit_post):
            res = rq.get(_next, params=params)
            try:
                _next = res.json()["paging"]["next"]
                yield {"status": True, "data": res.json()["data"]}
            except Exception:
                yield {
                    "status": False,
                    "data": [],
                    "message": res.json()["error"],
                    "next": _next,
                }
                if "blocked" in res.text or "occurred" in res.text or "password" in res.text:
                    idx_token += 1
                    try:
                        self.access_token = self.access_tokens[idx_token]
                        params["access_token"] = self.access_token
                    except:
                        tk = input("Access token=")
                        self.access_token = tk
                        params["access_token"] = tk

    def get_comments(self, idpost, limit_comment=5000):
        res = rq.get(
            self.API_POST_COMMENTS.format(idpost),
            params={
                "access_token": self.access_tokens[0],
                "limit": limit_comment,
                "fields": """
                message,
                created_time,
                from,
                reactions.summary(true).limit(0).as(reactions)
            """,
            },
        )

        try:
            return res.json()["data"]
        except Exception:
            return res.json()["error"]

    def parse(self, x):
        return {
            "id": x["from"]["id"],
            "name": x["from"]["name"],
            "id_post": x["id"],
            "created_time": x["created_time"],
            "updated_time": x["updated_time"],
            "message": x["message"],
            "type": x["type"],
            "picture": x["picture"] if "picture" in x.keys() else "",
            "source": x["source"] if "source" in x.keys() else "",
            "properties": x["properties"] if "properties" in x.keys() else "",
            "like": x["like"]["summary"]["total_count"],
            "love": x["love"]["summary"]["total_count"],
            "haha": x["haha"]["summary"]["total_count"],
            "wow": x["wow"]["summary"]["total_count"],
            "sad": x["sad"]["summary"]["total_count"],
            "angry": x["angry"]["summary"]["total_count"],
            "comments": [
                {
                    "created_time": comment["created_time"],
                    "message": comment["message"],
                    "id": comment["id"],
                    "reactions": comment["reactions"]["summary"]["total_count"],
                    "from": comment["from"],
                }
                for comment in x["comments"]["data"]
            ],
            "total_reactions": x["like"]["summary"]["total_count"]
                + x["love"]["summary"]["total_count"]
                + x["haha"]["summary"]["total_count"]
                + x["wow"]["summary"]["total_count"]
                + x["sad"]["summary"]["total_count"]
                + x["angry"]["summary"]["total_count"],
            "total_shares": x["shares"]["count"] if 'shares' in x.keys() else 0,
            "total_comments": x["comments"]["summary"]["total_count"]
        }

    def parse_data(self, data):
        return list(map(self.parse, data))

### Chuẩn bị thu thập dữ liệu

In [3]:
FANPAGE_ID = 'Biettheeodilam'

In [10]:
cookie_path = './cookies.json'
tokens = [
	'EAAD6V7os0gcBOZBoXDFQ1KKfQgVsqjODeaSUJHZAGnZBEp8IbReB25qhlnZC1lkgmAeZAe7ZBjK9AfiZCeqLVgPDd1LgQxCEqqRZCssUV1x4uYk6W272RyprUZAimuZAY0qs8do32W83ubkGYFMER5RT2o7lzxVCF2Jlzzuf1VtgPYKY2usZAY169tNVGjxLP49ZA9Dmx1ZBytar2XQZDZD',
	'EAAD6V7os0gcBOyE1vNjuZBMVeFZCP0QsOiFFJkRtZCZBy2ZAYWnJRZCyj8sN7q3spLfho6yNtLN12ZC7ugVkHXqgx5W2rtHsQUutdibPCBLxQOxDuNPZAQDfFgIOJywOHZCJMOjLRPSSw4Am5VOYrLFD7DDSfUb3XCcVe3NwPuOZA6FBAjrOMSgsRxvZCzw9H5uFIzm964kHogLWgZDZD',
	'EAAD6V7os0gcBO7VnDIx6TNdQLWcAwZCr0Vwo4CZBvV4cKZAgnrCe0wVK5vT0gWEvBmrTKBbL9I3FW5chv0Mcnp9zBXgfZBzb91MuyAopqI5gaQU0bvwIw5WfFwEZCEJUI62sOzvFG8nDjZC2hZBqCSQOEmkIBtKPoJeu9tLb11ZAQdADOa9bft0nfyWY8ZAQsT0FmmKKfb0MCmAZDZD',
	'EAAD6V7os0gcBOzHWJwh36XZC1h6Jq7KUpDpWP9H05c4FH0OV7mzdqcNkWvvuTXbPWXjrzSdc7NpIHFKnNSInjofgupHAZAQISTfb53WyI3r2cYUPnbjuT8XpJ6iyqHjsZA5y4kc6UxxZABzQ3DJ2JwR5HloTsc7gEQKlL1BwwL6ZBr3ZCjAHZCtJB9IAiZBhmGQVtjcwN41rowZDZD',
	'EAAD6V7os0gcBO5hxosSH62zPAqDDbY0df9Ot4ykhZBpSfZAZCuGgqTC3stH2oUEke8dVoirbKvkISIl0kVipast4BuZB0fTxp6vflDJUka9FSgg4kmVxMeN4ax9tBuikFJ8fQC8wGDnR3Uo40qLXVjuy3yWWPrTL59Ifd4tTmjLltW9mM0dnwsuohhCLaJA2fHtCkoNPpwZDZD',
	'EAAD6V7os0gcBO2yMZAKfDPcYYJyKp1p1e6Gwsabiwp6lNvaGkyTPT25pmLcN7TkHd2Qbm3pYGfKtz0oqK0xau5for3nEEdNNp3oWpOYWhbmptVwrUL8bbTzeuobTcDTFWebca0NWBpFOVoateZBk4SZBfwhhZAEZBzqmzjQiyASwqM8PBqnFsLQf4FiLZCZAh4taEhu1ZC1AWQZDZD',
	'EAAD6V7os0gcBO5ZAgPbZB7aZCIWAz77JZC0eatuQZA5GZAZAeddfkTRYkUTrEDZCuJ4dw7UYucYLJn0XrlFFW16BG76T3XoV0aAL2AKgdeGbJQOuYQ4n7v2DsakuHtmbZAZBsQ2m3gv2QWOpz2kbnxE7LgnHkzYj2gpSxZA8SX19QlQSIE5kXujwcrTP0gpYPheCZBmEywFPnMTWxQZDZD',
	'EAAD6V7os0gcBO1JuzPBOYzsJPUyleYPX0ZBTvHSpUqsZB3rxsVLdaGzaykR11dbWQKEoyy1W6XRy67nG4qDoslt5ZAoIv887UJiRI0vV7xw0Hh0fkerWmR2J8JpJBYfC4IClt20DrAlxHy4WyeNNeiPSfEfftqkJYR0YOS12cFqjYcLCLEtAOcv9OgEVWg3Bk7mvQubTQZDZD'
]
fb_spider = FacebookCrawler(cookie_path=cookie_path, access_tokens=tokens, page_url=FANPAGE_ID)

### Thu thập và tái cấu trúc

##### 1. Lấy 600 post gần nhất: Thời gian(11:30:00 18/11/2023)

In [11]:
data = []
ids = []
for res in fb_spider.get_page_posts(limit_post=600, limit_comment=500):
	res_data = res["data"]
	status = res["status"]
	if status:
		for i in res_data:
			if i['id'] not in ids:
				data.append(i)
				ids.append(i['id'])
			print(i['id'], "total:", len(ids))
	else:
		print(res["message"], res["next"])

101084031778928_915132193952422 total: 1
101084031778928_915132193952422 total: 1
101084031778928_915087943956847 total: 2
101084031778928_915087943956847 total: 2
101084031778928_914694960662812 total: 3
101084031778928_914694960662812 total: 3
101084031778928_914645237334451 total: 4
101084031778928_914645237334451 total: 4
101084031778928_914608167338158 total: 5
101084031778928_914608167338158 total: 5
101084031778928_914527687346206 total: 6
101084031778928_914527687346206 total: 6
101084031778928_914390140693294 total: 7
101084031778928_914390140693294 total: 7
101084031778928_914050984060543 total: 8
101084031778928_914050984060543 total: 8
101084031778928_913913880740920 total: 9
101084031778928_913913880740920 total: 9
101084031778928_913972200735088 total: 10
101084031778928_913972200735088 total: 10
101084031778928_913950664070575 total: 11
101084031778928_913950664070575 total: 11
101084031778928_913900767408898 total: 12
101084031778928_913900767408898 total: 12
1010840317

In [12]:
print("Số lượng bài viết cào được:", len(data))

Số lượng bài viết cào được: 598


In [13]:
print("Bài đăng xa nhất thu thập được: {0} - Thời gian đăng bài: {1}".format(data[-1]['id'], data[-1]['created_time']))

Bài đăng xa nhất thu thập được: 101084031778928_852735896858719 - Thời gian đăng bài: 2023-08-21T09:06:56+0000


##### 2. Lưu dữ liệu thô vào file json

In [14]:
fb_spider.export_to_csv(data=data, file_name="598_rawdata.csv")

True

In [16]:
df = pd.read_csv("./Data/598_rawdata.csv")
df.shape

(598, 15)

##### 3. Tái tổ chức dữ liệu

In [17]:
# Dữ liệu thô post đầu tiên thu thập được
data[0]

{'created_time': '2023-11-18T04:13:09+0000',
 'from': {'name': 'Biết thế éo đi làm', 'id': '101084031778928'},
 'updated_time': '2023-11-18T04:30:55+0000',
 'message': 'Một ngày làm việc hiệu quả của người lương 5 triệu',
 'type': 'photo',
 'picture': 'https://scontent.fhan3-5.fna.fbcdn.net/v/t39.30808-6/401837213_915132177285757_702516156832728207_n.jpg?stp=cp1_dst-jpg_p130x130&_nc_cat=109&ccb=1-7&_nc_sid=5f2048&_nc_ohc=3RUOpLcrl_kAX-VfcuW&_nc_ad=z-m&_nc_cid=0&_nc_ht=scontent.fhan3-5.fna&cb_e2o_trans=t&oh=00_AfDspdgVcVXYv0v5bLNOQZh5S4p2dw-yMknsnqdAWzwpsw&oe=655C417D',
 'like': {'data': [],
  'summary': {'total_count': 293, 'viewer_reaction': 'NONE'}},
 'love': {'data': [],
  'summary': {'total_count': 3, 'viewer_reaction': 'NONE'}},
 'haha': {'data': [],
  'summary': {'total_count': 204, 'viewer_reaction': 'NONE'}},
 'wow': {'data': [], 'summary': {'total_count': 0, 'viewer_reaction': 'NONE'}},
 'angry': {'data': [],
  'summary': {'total_count': 0, 'viewer_reaction': 'NONE'}},
 'sad':

In [18]:
data_restructure = fb_spider.parse_data(data)

In [19]:
# Dữ liệu post đầu tiên sau khi được tái cấu trúc
data_restructure[0]

{'id': '101084031778928',
 'name': 'Biết thế éo đi làm',
 'id_post': '101084031778928_915132193952422',
 'created_time': '2023-11-18T04:13:09+0000',
 'updated_time': '2023-11-18T04:30:55+0000',
 'message': 'Một ngày làm việc hiệu quả của người lương 5 triệu',
 'type': 'photo',
 'picture': 'https://scontent.fhan3-5.fna.fbcdn.net/v/t39.30808-6/401837213_915132177285757_702516156832728207_n.jpg?stp=cp1_dst-jpg_p130x130&_nc_cat=109&ccb=1-7&_nc_sid=5f2048&_nc_ohc=3RUOpLcrl_kAX-VfcuW&_nc_ad=z-m&_nc_cid=0&_nc_ht=scontent.fhan3-5.fna&cb_e2o_trans=t&oh=00_AfDspdgVcVXYv0v5bLNOQZh5S4p2dw-yMknsnqdAWzwpsw&oe=655C417D',
 'source': '',
 'properties': '',
 'like': 293,
 'love': 3,
 'haha': 204,
 'wow': 0,
 'sad': 6,
 'angry': 0,
 'comments': [{'created_time': '2023-11-18T04:13:24+0000',
   'message': 'Người đi làm thứ 7 là người khổ nhất',
   'id': '915132193952422_363754546106175',
   'reactions': 15,
   'from': {'id': '101084031778928',
    'name': 'Biết thế éo đi làm',
    'birthday': '02/24/1986',

##### 4. Lưu dữ liệu đã tái cấu trúc

In [20]:

fb_spider.export_to_csv(data_restructure, '598_parsedata.csv')

True

### Kiểm tra dữ liệu

In [21]:
df_raw = pd.read_csv('./Data/598_parsedata.csv')
df_raw.head()

,id,name,id_post,created_time,updated_time,message,type,picture,source,properties,like,love,haha,wow,sad,angry,comments,total_reactions,total_shares,total_comments
0,101084031778928,Biết thế éo đi làm,101084031778928_915132193952422,2023-11-18T04:13:09+0000,2023-11-18T04:30:55+0000,Một ngày làm việc hiệu quả của người lương 5 t...,photo,https://scontent.fhan3-5.fna.fbcdn.net/v/t39.3...,NaN,NaN,293,3,204,0,6,0,"[{'created_time': '2023-11-18T04:13:24+0000', ...",506,3,19
1,101084031778928,Biết thế éo đi làm,101084031778928_915087943956847,2023-11-18T02:16:00+0000,2023-11-18T04:21:21+0000,Khách vote 1 ⭐ vì nhắn lúc 3h sáng không nhân ...,status,NaN,NaN,NaN,310,0,434,1,18,0,"[{'created_time': '2023-11-18T02:17:04+0000', ...",763,9,47
2,101084031778928,Biết thế éo đi làm,101084031778928_914694960662812,2023-11-17T13:39:28+0000,2023-11-18T04:01:51+0000,"""Em yên tâm, chỉ có chị với bé Linh biết chuyệ...",status,NaN,NaN,NaN,1023,3,1128,0,14,0,"[{'created_time': '2023-11-17T13:41:25+0000', ...",2168,19,110
3,101084031778928,Biết thế éo đi làm,101084031778928_914645237334451,2023-11-17T12:15:30+0000,2023-11-18T02:48:05+0000,"Người sếp khi nghe ứng viên nói: ""Lương không ...",photo,https://scontent.fhan3-2.fna.fbcdn.net/v/t39.3...,NaN,NaN,685,4,485,2,6,0,"[{'created_time': '2023-11-17T12:16:06+0000', ...",1182,8,46
4,101084031778928,Biết thế éo đi làm,101084031778928_914608167338158,2023-11-17T11:14:43+0000,2023-11-18T04:02:28+0000,Thế giới đang chia ra thành 2 loại người: Mai ...,status,NaN,NaN,NaN,755,2,310,3,105,1,"[{'created_time': '2023-11-17T16:23:12+0000', ...",1176,37,147
